# Make Sense Backend on Google Colab

Gunakan notebook ini untuk menjalankan backend Make Sense (YOLOv8) di Google Colab. 
Ini memungkinkan Anda memanfaatkan GPU gratis Google untuk inferensi yang lebih cepat.

### Langkah-langkah:
1.  Jalankan sel instalasi dependensi.
2.  Upload model YOLO custom Anda (`best.pt`) ke file storage Colab (di panel kiri).
3.  Masukkan Authtoken Ngrok Anda (Daftar gratis di [ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken)).
4.  Jalankan server.
5.  Salin URL publik (misal: `https://xxxx-xx-xx-xx-xx.ngrok-free.app`) dan masukkan ke frontend Make Sense.

In [ ]:
# 1. Install Dependencies
!pip install fastapi uvicorn python-multipart ultralytics pyngrok nest-asyncio

In [ ]:
# 2. Define API & App (Main Code)
import os
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.middleware.cors import CORSMiddleware
from ultralytics import YOLO
from PIL import Image
import io
import numpy as np
import cv2

app = FastAPI()

# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins for Colab usage
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load YOLO model
# PASTIKAN ANDA SUDAH MENGUPLOAD 'best.pt' KE FILE COLAB (/content/best.pt)
model_path = "/content/best.pt"
try:
    if os.path.exists(model_path):
        model = YOLO(model_path)
        print("YOLO model loaded successfully!")
    else:
        model = None
        print(f"Model not found at {model_path}. Please upload 'best.pt'. Using standard yolov8n.pt as fallback.")
        model = YOLO("yolov8n.pt")
except Exception as e:
    print(f"Error loading YOLO model: {e}")
    model = None

@app.get("/")
async def root():
    return {"status": "online", "message": "Make Sense Backend is running on Colab!"}

@app.post("/annotate")
async def annotate_image(file: UploadFile = File(...)):
    if not model:
        return {"success": False, "error": "Model not loaded"}

    try:
        # Read image file
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("RGB")
        img_array = np.array(image)

        # Run inference
        results = model(img_array)

        # Process results
        annotations = []
        for result in results:
            boxes = result.boxes.cpu().numpy()
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = box.xyxy[0]
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                try:
                    class_name = model.names[class_id]
                except:
                    class_name = str(class_id)

                # Calculate bbox format [center_x, center_y, width, height] used by Frontend
                w = x2 - x1
                h = y2 - y1
                x = x1 + w / 2
                y = y1 + h / 2

                annotations.append({
                    "bbox": [float(x), float(y), float(w), float(h)],
                    "class": class_name,
                    "score": confidence
                })

        return {
            "success": True,
            "annotations": annotations,
            "count": len(annotations)
        }

    except Exception as e:
        print(f"Error processing image: {e}")
        return {"success": False, "error": str(e)}

In [ ]:
# 3. Run Server with Ngrok
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

# --- CONFIGURATION ---
# Masukkan token Ngrok Anda di sini
NGROK_AUTH_TOKEN = "MASUKKAN_TOKEN_ANDA_DISINI"
# ---------------------

if NGROK_AUTH_TOKEN == "MASUKKAN_TOKEN_ANDA_DISINI":
    print("⚠️ PERINGATAN: Anda belum memasukkan Ngrok Auth Token.")
    print("Silakan daftar di https://dashboard.ngrok.com/get-started/your-authtoken dan tempel token Anda di atas.")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Kill existing tunnels
    ngrok.kill()

    # Open a HTTP tunnel on port 8000
    public_url_obj = ngrok.connect(8000)
    public_url = public_url_obj.public_url
    
    print("="*50)
    print(f"🚀 PUBLIC URL ANDA: {public_url}")
    print("Copy URL di atas dan paste ke 'Backend URL' di popup Make Sense.")
    print("="*50)

    # Run the server
    uvicorn.run(app, port=8000)